# 元编程（Meta-Programming）

什么是元编程呢？

简而言之，代码是可以操作的对象，我可以通过元编程来**自动生成代码， 并执行代码**

## 表达式

### 通过字符串创建表达式

In [1]:
prog = "1+1"

"1+1"

In [2]:
ex1 = Meta.parse(prog)

:(1 + 1)

In [3]:
eval(ex1)

2

In [4]:
dump(ex1)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Int64 1
    3: Int64 1


### 通过`Expr`手动创建

In [5]:
Expr(:call, :+, 1, 1)

:(1 + 1)

### 通过`quote`创建多个表达式

In [6]:
ex3 = quote
    x = 1
    y = 2
end

quote
    #= In[6]:2 =#
    x = 1
    #= In[6]:3 =#
    y = 2
end

In [7]:
dump(ex3)

Expr
  head: Symbol block
  args: Array{Any}((4,))
    1: LineNumberNode
      line: Int64 2
      file: Symbol In[6]
    2: Expr
      head: Symbol =
      args: Array{Any}((2,))
        1: Symbol x
        2: Int64 1
    3: LineNumberNode
      line: Int64 3
      file: Symbol In[6]
    4: Expr
      head: Symbol =
      args: Array{Any}((2,))
        1: Symbol y
        2: Int64 2


可以看出`quote`通过`array`嵌套了多个表达式， 他的`head`是`:block`（中间有两行是注释）

In [8]:
esc(ex3)

:($(Expr(:escape, quote
    #= In[6]:2 =#
    x = 1
    #= In[6]:3 =#
    y = 2
end)))

### 直接通过`:()`创建

In [9]:
:(1+1) == Expr(:call, :+, 1, 1)

true

In [10]:
ex2 = :(2 + 3 * a + b) 

:(2 + 3a + b)

In [11]:
dump(ex2)

Expr
  head: Symbol call
  args: Array{Any}((4,))
    1: Symbol +
    2: Int64 2
    3: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol *
        2: Int64 3
        3: Symbol a
    4: Symbol b


julia的表达式通常尽可以包含`符号、子表达式、字面值`， 上式中`+, b`是符号， `:(3 * a)`是子表达式， `2`是字面值，上式是一个嵌套的表达式， **`a, b`的值还不知道， 在这里被当做符号对待**

<font style="color:purple;font-size:14pt;">如果`a, b`已知， 那么通过`$`进行插值， 可以使构造的表达式中的`a, b`替换为变量`a, b`的实际值</font>

In [12]:
a = 1
b = 1
:(2 + 3 * a + b) # no $

:(2 + 3a + b)

In [13]:
# 仍然是可以求值的
eval(:(2 + 3 * a + b))

6

In [14]:
:(2 + 3 * $a + $b) #！！ with $ 

:(2 + 3 * 1 + 1)

In [15]:
# 很自然的可以求值
eval(:(2 + 3 * $a + $b))

6

In [16]:
# splatting interploration

args = [:x, :y, :z]
:(f(1, $(args...)))

:(f(1, x, y, z))

可以看出来整个工作的流程就是， 利用`Expr`表达式生成代码， 然后使用`eval`函数求值， 达到自动生成代码， 运行代码的目的

## 宏

### 定义宏

In [17]:
macro sayhello()
    return :(println("Hello World!"))
end

@sayhello (macro with 1 method)

In [18]:
@sayhello()
@sayhello

Hello World!
Hello World!


通过`@macro`执行宏代码时， 先会生成`Expr`表达式， 然后自动使用`eval`函数求值

`@macroexpand`可以看到生成的`Expr`表达式

In [19]:
macro sayhello(name)
    return :(println("Hello World! ", $name))
end

@sayhello (macro with 2 methods)

In [20]:
@sayhello "XJZ"

Hello World! XJZ


In [21]:
@macroexpand @sayhello "XJZ"

:(Main.println("Hello World! ", "XJZ"))

注意`sayhello(name)`这个宏中用到了插值`$name`, 为啥呢？可以这么理解， 在函数体内部， `name`是一个局部变量， **如果不插值， 那么在函数外部的作用域中执行宏时，不知道`name`的值**

下方的函数作为一个对比， 可以看出`name`是一个局部变量

In [22]:
macro sayhello2(name)
    return :(println("Hello World! ", name))
end

@sayhello2 "XJZ"

LoadError: UndefVarError: name not defined

### 理解宏的运行

（1） 宏如何运行？

In [23]:
macro twostep(arg...)
    println("I execute at parse time. The argument is: ", arg...)
    return :(println("I execute at runtime. The argument is: ", $(arg...)))
end

@twostep (macro with 1 method)

In [24]:
@macroexpand @twostep(1, 2, 3)

I execute at parse time. The argument is: 123


:(Main.println("I execute at runtime. The argument is: ", 1, 2, 3))

可以看出， 仅生成表达式时， 会执行中间定义的命令

执行宏命令时， 指定定义在宏内部的命令， 以及`eval`返回的表达式的值

In [25]:
@twostep 1, 2, 3

I execute at parse time. The argument is: (1, 2, 3)
I execute at runtime. The argument is: (1, 2, 3)


（2） 作用域如何区别？

当宏内部定义的一些变量与宏运行环境的一些变量名字一样时， 怎么办？

In [26]:
# 局部变量， 运行环境内的变量没有任何影响
x = 0.1
y = 0.2
macro envtest1()
    return quote
         x = 1
         y = 2
    end
end
@envtest1
x, y

(0.1000, 0.2000)

In [27]:
# esc(), escape逃离这种机制， 生成的代码会在宏运行的环境内运行
x = 0.1
y = 0.2
macro envtest1()
    expr = quote
         x = 1
         y = 2
    end
    return esc(expr)
end
@envtest1
x, y

(1, 2)

<font style="color:purple;font-size:14pt">这个地方我的理解好像还有点浅显， 先这种用着， 官方文档里的例子在开发Julia包时才能用得上</font>

### 案例一：assert

In [28]:
macro assert(ex)
    return :($ex ? nothing : AssertionError($ex))
end

@assert (macro with 1 method)

In [29]:
@assert 1 == 2.0

LoadError: MethodError: no method matching convert(::Type{AbstractString}, ::Bool)
[0mClosest candidates are:
[0m  convert([91m::Type{Any}[39m, ::Any) at C:\Users\XJZ\AppData\Local\julias\julia-1.7\share\julia\base\boot.jl:445
[0m  convert(::Type{T}, [91m::T[39m) where T at C:\Users\XJZ\AppData\Local\julias\julia-1.7\share\julia\base\boot.jl:446

In [30]:
@macroexpand @assert 1 == 2.0

:(if 1 == 2.0000
      Main.nothing
  else
      Main.AssertionError(1 == 2.0000)
  end)

### 案例二：Fibonacci数列

$$a_0 = 1$$
$$a_1 = 1$$
$$a_i = a_{i-1} + a_{i-2}, \text{if}\ i \geq 2$$

**（1）传统思路：查表**

In [31]:
function fib1(n)
    rst = zeros(n)
    if n == 1
        return 1.0
    elseif n == 2
        return 1.0
    elseif n > 2 
        rst[1:2] .= 1
        for i in 3:length(rst)
            rst[i] = rst[i - 1] + rst[i - 2]
        end
        return rst[end]
    end 
end

fib1 (generic function with 1 method)

In [32]:
using BenchmarkTools

In [33]:
@btime fib1(10)

  32.931 ns (1 allocation: 144 bytes)


55.0000

以上实现了一个可以查表的fibnacci数列的计算方面，避免了递归， 运算性能还不错， 几乎是线性复杂度， 他的原理是**程序在运行时创建一个向量， 存储之前的计算结果， 后续计算可以利用之前的计算结果**

**（2）编译期查表**

还不太会`@generated`